In [1]:
# Handwritten Neural Network test: HandwrittenNN.ipynb
# Author: comtalyst

In [30]:
import numpy as np
import matplotlib.pyplot as plot

In [52]:
def sigmoid(Z):
    A = 1/(1 + np.exp(-Z))
    return A

def sigmoid_deriv(Z):
    '''
    (Coursera DL Notes, p. 8)
    '''
    A = sigmoid(Z)
    return np.multiply(A,(1 - A))

In [53]:
def relu(Z):
    A = np.max(0, Z)
    return A

def relu_deriv(Z):
    '''
    (Coursera DL Notes, p. 9)
    '''
    if Z < 0:
        return 0
    else:
        return 1

In [80]:
def forward_propagation(X, W, b, activations):
    L = W.shape[0]          # layers
    n = X.shape[0]          # features
    m = X.shape[1]          # samples

    # initialize linear output
    '''
    This np.ndarray allow us to contruct an array with size initialized and can have any data type in it (from dtype=object)
    Therefore, we could use this to create histogram-like array for the uneven NN
    '''
    Z = np.ndarray(shape=[L+1], dtype=object)
    A = np.ndarray(shape=[L+1], dtype=object)
    
    # base case
    Z[1] = np.dot(W[1], X) + b[1]
    # activation
    if activations[0] == "sigmoid":
        A[1] = sigmoid(Z[1])
    elif activations[0] == "relu":
        A[1] = relu(Z[1])
    else:
        A[1] = Z[1]

    # loop the layers
    for l in range(2, L):
        Z[l] = np.dot(W[l], A[l-1]) + b[l]
        if activations[l-1] == "sigmoid":
            A[l] = sigmoid(Z[l])
        elif activations[l-1] == "relu":
            A[l] = relu(Z[l])
        else:
            A[l] = Z[l]

    return A, Z
        

In [79]:

L = 2
n1 = 3
n2 = 4
#n3 = 2
n = 2
m = 5
activations = ["dumb", "dumb", "dumb"]

X = np.round(np.random.rand(n, m)*10) % 10
W = np.ndarray(shape=[L+1], dtype=object)
b = np.ndarray(shape=[L+1], dtype=object)

W[1] = np.round(np.random.rand(n1, n)*10) % 10
W[2] = np.round(np.random.rand(n2, n1)*10) % 10
#W[3] = np.round(np.random.rand(n3, n2)*10) % 10
b[1] = np.round(np.random.rand(n1, 1)*10) % 10
b[2] = np.round(np.random.rand(n2, 1)*10) % 10
#b[3] = np.round(np.random.rand(n3, 1)*10) % 10

A, Z = forward_propagation(X, W, b, activations)
print("X")
print(X)
print("W")
print(W)
print("b")
print(b)
print("A")
print(A)

X
[[1. 4. 9. 7. 6.]
 [7. 1. 0. 9. 2.]]
W
[None array([[8., 0.],
       [1., 2.],
       [6., 4.]])
 array([[7., 1., 7.],
       [9., 1., 6.],
       [1., 5., 7.],
       [7., 3., 6.]])]
b
[None array([[4.],
       [2.],
       [3.]])
 array([[6.],
       [4.],
       [3.],
       [9.]])]
A
[None
 array([[12., 36., 76., 60., 52.],
       [17.,  8., 11., 27., 12.],
       [37., 31., 57., 81., 47.]])
 array([[ 366.,  483.,  948., 1020.,  711.],
       [ 351.,  522., 1041., 1057.,  766.],
       [ 359.,  296.,  533.,  765.,  444.],
       [ 366.,  471.,  916.,  996.,  691.]])
 None]
